In [13]:
import pandas as pd
import plotly.express as px
import warnings 
from internproject.constants import ROOT_DATA_DIR

add_lp = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_add_lp.parquet")  # more details on the add liquidity events
remove_lp = pd.read_parquet(
    ROOT_DATA_DIR / "stETH_ETH_remove_lp.parquet"
)  # more details on the remove liquidity events
lp_transfers = pd.read_parquet(ROOT_DATA_DIR / "stETH_ETH_lp_transfers.parquet")

In [109]:
stETH_ETH_apr_df = pd.read_csv(
    ROOT_DATA_DIR
    / 'daily_snapshots/0xDC24316b9AE028F1497c275EB9192a3Ea0f67022_apr_df.csv',
    index_col=0,
    parse_dates=True,
).dropna()

# finding rolled avg apr from og steth eth dataframe and set it as price 
stETH_ETH_apr_df['cur_naive_apr'] = stETH_ETH_apr_df[['raw_base_apr', 'raw_fee_apr', 'raw_incentive_apr', 'raw_price_apr']].sum(axis=1)
stETH_ETH_apr_df['rolling_cur_naive_apr'] = stETH_ETH_apr_df['cur_naive_apr'].rolling(7).mean()
stETH_ETH_apr_df['rolling_cur_naive_apr'].median()

5.086284024877622

In [3]:
null_address = "0x0000000000000000000000000000000000000000"

### Understanding lp transfers

1. The "value" column in `lp_transfers` refers to the quantity of lp tokens
2. If the null address is in the "from" column, then it is adding liquidity to the pool. 
3. If the null address is in the "to" column, then it is removeing liquidity from the pool.
4. It is safe to ignore the other transfers

In [4]:
add_liquidity_transfers = lp_transfers[lp_transfers["from"] == null_address]
withdraw_liquidity_transfers = lp_transfers[lp_transfers["to"] == null_address]

In [5]:
withdraw_liquidity_transfers

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-05 17:12:33,0x3a63aA8AE11Ecaa82fC352d0390634E8030B2ffd,0x0000000000000000000000000000000000000000,2.000945,Transfer,11595786,133,295,0xbd58a23237562e0332aaa7a7668f8dd71e4aecddc95d...,2021-01-05 17:12:33
2021-01-05 17:46:38,0xaF297deC752c909092A117A932A8cA4AaaFF9795,0x0000000000000000000000000000000000000000,499.501433,Transfer,11595955,11,5,0xad790ed2cb537281ae0c7eaf1558b1494b869e7f7e1c...,2021-01-05 17:46:38
2021-01-05 21:03:54,0xb084F0AdB86BF6B001E540Ac05439D98A499ce1D,0x0000000000000000000000000000000000000000,0.999754,Transfer,11596894,75,96,0xc485ceb5703ddaf5393049b8fb71a06b20ee0c0b4ced...,2021-01-05 21:03:54
2021-01-06 03:36:23,0x89B8B20AE90328692cD367f75aaFadF55fd33E8B,0x0000000000000000000000000000000000000000,10.037649,Transfer,11598604,77,130,0xbebdcd6fa25430bea017919704fd3dcd12f55ef0f99b...,2021-01-06 03:36:23
2021-01-06 12:17:18,0xdE3E412d4fe3c9d90ac74d0A9B064951B39EEae4,0x0000000000000000000000000000000000000000,1.002055,Transfer,11600957,56,66,0x5a33f4f9e2e2a55f09141909f75b1b308d092183681c...,2021-01-06 12:17:18
...,...,...,...,...,...,...,...,...,...
2023-06-20 22:25:35,0xEa508F82728927454bd3ce853171b0e2705880D4,0x0000000000000000000000000000000000000000,0.000000,Transfer,17523858,45,95,0x6188d0177bb9a8a4a3bb5feb560477d6ec4642c9fde0...,2023-06-20 22:25:35
2023-06-20 22:33:11,0x46D5F548E02fdee3ea7E4168B7cF896429fE6fe7,0x0000000000000000000000000000000000000000,65.033669,Transfer,17523896,78,240,0x113260903e90d948871a8a0af2f03156c17781a455ea...,2023-06-20 22:33:11
2023-06-20 22:40:35,0x5aD15EBa9D4E9351414084Eb1dd1EACc2068A4A7,0x0000000000000000000000000000000000000000,51.167100,Transfer,17523933,42,101,0x39af71a3003ae573c6739b9679bede548ea7cc53a6b0...,2023-06-20 22:40:35


### Analyzing behavior and liquidity of unique users 
- combining the added and withdrawed liquidty 

In [14]:
withdraw_liquidity_transfers['value'] = withdraw_liquidity_transfers['value']* -1

warnings.filterwarnings('ignore')
withdraw_liquidity_transfers['value']

timestamp
2021-01-05 17:12:33     -2.000945
2021-01-05 17:46:38   -499.501433
2021-01-05 21:03:54     -0.999754
2021-01-06 03:36:23    -10.037649
2021-01-06 12:17:18     -1.002055
                          ...    
2023-06-20 22:25:35     -0.000000
2023-06-20 22:33:11    -65.033669
2023-06-20 22:40:35    -51.167100
2023-06-20 23:28:11     -0.000000
2023-06-21 00:00:11     -0.003230
Name: value, Length: 27851, dtype: float64

In [15]:
add_liquidity_transfers['user'] = add_liquidity_transfers['to']
withdraw_liquidity_transfers['user'] = withdraw_liquidity_transfers['from']

transfer_df = pd.concat([add_liquidity_transfers, withdraw_liquidity_transfers])
transfer_df.columns

warnings.filterwarnings('ignore')

In [16]:
# locating one unique user
transfer_df['user'].value_counts().head(100).tail(1)

0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D    25
Name: user, dtype: int64

In [17]:
transfer_df['user'] == '0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D'

timestamp
2021-01-05 00:26:18    False
2021-01-05 11:26:21    False
2021-01-05 13:35:18    False
2021-01-05 13:48:47    False
2021-01-05 14:32:47    False
                       ...  
2023-06-20 22:25:35    False
2023-06-20 22:33:11    False
2023-06-20 22:40:35    False
2023-06-20 23:28:11    False
2023-06-21 00:00:11    False
Name: user, Length: 62068, dtype: bool

In [18]:
transfer_df_1person = transfer_df[transfer_df['user'] == '0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D']

In [19]:
# finding cumulative sum of this person's transfer payments, which is the same as their balance over time
transfer_df_1person['value'].cumsum().round(3).values
transfer_df_1person['balance_over_time'] = transfer_df_1person['value'].cumsum().round(3).values

warnings.filterwarnings('ignore')

In [20]:
# amount of tokens supplied (balance) over time 
px.bar(transfer_df_1person['value'].cumsum().round(3).values)

In [254]:
# repeating process above for all unique users in transfer df to FIND WHALES
transfer_df.sort_index(inplace=True)
transfer_df['cumsol_col'] = transfer_df.groupby(['user'])['value'].cumsum()
pivot_df = transfer_df.pivot_table(values='cumsol_col', index=transfer_df.index, columns='user')


NameError: name 'transfer_df' is not defined

In [14]:
pivot_df = pivot_df.ffill()

In [70]:
px.line(pivot_df['0xb76edb86C89f241aE87CE7fE9e48E10588CA1D0D'])

NameError: name 'pivot_df' is not defined

### when did this person enter and leave

In [6]:
example_person = "0x80ee7c0e1e59929823eD9B22e34538b226967109"
add_liquidity_transfers[add_liquidity_transfers["to"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-12 22:16:32,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,1.999396,Transfer,11642841,30,51,0xb03636820959b9641a5c4c39676967180d7629a9bc30...,2021-01-12 22:16:32
2021-01-12 22:31:11,0x0000000000000000000000000000000000000000,0x80ee7c0e1e59929823eD9B22e34538b226967109,0.999698,Transfer,11642902,120,187,0xcf8723e0451638d3f68a12395facc8c0f1ed2178a90f...,2021-01-12 22:31:11


In [7]:
withdraw_liquidity_transfers[withdraw_liquidity_transfers["from"] == example_person]

,from,to,value,event,block,transaction_index,log_index,hash,timestamp
timestamp,,,,,,,,,
2021-01-14 13:13:07,0x80ee7c0e1e59929823eD9B22e34538b226967109,0x0000000000000000000000000000000000000000,2.999095,Transfer,11653414,210,243,0x642a0a0273035551b6fa585c8615e129bc05772c9277...,2021-01-14 13:13:07


This person entered twice at 2021-01-12 and then left at 2021-01-14. So they were only in for 3 days.

### Open Problems

Here are some problems you could look at. Remember that the focus is figuring out how changing incentive APR affacts LPs entering and leaving the pool

1. How does the amount of ETH value added influence the duration of a person's stay in the pool? (does this relationship change over time?)
2. Among those who fully exited the pool, what was the difference between the starting and ending Annual Percentage Rate (APR)?
3. What was the average duration of people's participation in the pool?
4. How frequently did people perform actions such as adding liquidity or withdrawing liquidity? Did they break up these actions into smaller increments?
5. Can the participants be clustered into meaningful groups based on their LP sizes and durations?
6. Are there any notable thresholds or breakpoints at which people exhibit different behaviors? eg. after a person has been in a pool for > 1 month they almost never leave
7. It is important to pay special attention to participants with large LPs. How do the people who have added and withdrawn large amounts of liquidity differ?

# Elasticity Subgroups

##### LP's who provided over 100 ETH 

In [207]:
# merge add_lp with stETH_ETH_apr_df so that we can access the apr's 
add_lp['date'] = pd.to_datetime(add_lp['timestamp'].dt.date)
merged_df = pd.merge(add_lp, stETH_ETH_apr_df, left_on='date', right_on='timestamp', how='left') 
merged_df.set_index('date', inplace=True)

In [277]:
# sum up tokens provided for each coin and select lp's that provided over 100 
merged_df['total_tokens'] = merged_df['token_amounts_0'] + merged_df['token_amounts_1']  
add_over_100_subgroup = merged_df.loc[merged_df['total_tokens'] > 100] 

subgroup_shift = add_over_100_subgroup.shift(-30, freq='D')
add_over_100_subgroup = pd.merge_asof(add_over_100_subgroup, subgroup_shift[['rolling_cur_naive_apr', 'token_supply']], left_index=True, right_index=True, direction='forward', suffixes=('', '_prev'))

In [279]:
# set variables for elasticity formula
q2 = add_over_100_subgroup['token_supply']
q1 = add_over_100_subgroup['token_supply_prev']
p2 = add_over_100_subgroup['rolling_cur_naive_apr']
p1 = add_over_100_subgroup['rolling_cur_naive_apr_prev']

# calculate elasticity 
percent_change_supply = (q2 - q1) / ((q2 + q1) / 2) * 100 
percent_change_price = (p2 - p1) / ((p2 + p1) / 2) * 100 
add_over_100_subgroup['elasticity'] = percent_change_supply / percent_change_price 
add_over_100_subgroup['elasticity']

warnings.filterwarnings('ignore')

##### Mostly similar, but we're missing the large negative shock in May 2023 AND the positive shock is much larger 
- As apr increases, we expect quantity to increase, i.e. all elasticity values should be positive. This market seems to behave more rationally than the other market. 

In [308]:
px.line(add_over_100_subgroup['elasticity'].dropna())

In [311]:
px.line(add_over_100_subgroup['market_eth_tvl'].dropna())

- In regards to the large positive shock, there could be outside factors unrelated to apr that cause ppl to leave/enter. For example, crazy low apr in other markets would lead to lots of people entering stETH ETH market because they're attracted to the higher return. 
- other markets unsafe

#### LP's that added liquidity when APR is greater than median percentage

In [288]:
# convert to datetime so we don't lose data, then merge add_liquidity_transfers with stETH_ETH_apr_df SO WE CAN ACCESS APR'S 
add_liquidity_transfers['date_only'] = pd.to_datetime(add_liquidity_transfers.index.date)
merge2_df = pd.merge(add_liquidity_transfers, stETH_ETH_apr_df, left_on='date_only', right_on='timestamp', how='left')
merge2_df.set_index('date_only', inplace=True)


In [300]:
# find where lp's added liquidity when apr is greater than median 
added_when_apr_over_med = merge2_df.loc[merge2_df['rolling_cur_naive_apr'] > merge2_df['rolling_cur_naive_apr'].median()]
# added_when_apr_over_med = added_when_apr_over_med[~added_when_apr_over_med.index.duplicated(keep='first')]

shift = added_when_apr_over_med.shift(-30, freq='D')
added_when_apr_over_med = pd.merge_asof(added_when_apr_over_med, shift[['rolling_cur_naive_apr', 'value']], left_index=True, right_index=True, direction='forward', suffixes=('', '_prev'))


In [301]:
# set price variables 
p2 = added_when_apr_over_med['rolling_cur_naive_apr']
p1 = added_when_apr_over_med['rolling_cur_naive_apr_prev']

# set supply variables
q2 = added_when_apr_over_med['value']
q1 = added_when_apr_over_med['value_prev']

# calculate elasticity
percent_change_supply = (q2 - q1) / ((q2 + q1) / 2) * 100 
percent_change_price = (p2 - p1) / ((p2 + p1) / 2) * 100 
added_when_apr_over_med['elasticity'] = percent_change_supply / percent_change_price


In [305]:
px.line(added_when_apr_over_med['elasticity'].dropna())

######
This elasticity does not seem to even be influenced by apr. The infinitely large elasticity values could indicate perfect elasticity or error in the code 

In [303]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=added_when_apr_over_med.index, y=added_when_apr_over_med['elasticity'], name='elasticity'), 
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=added_when_apr_over_med.index, y=added_when_apr_over_med['rolling_cur_naive_apr'], name='price (apr)'), 
    secondary_y=True,
)

# Set x-axis title
fig.update_xaxes(title_text="time stamp")

# Set y-axes titles
fig.update_yaxes(title_text="<b>elasticity</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>price (measured in apr)</b>", secondary_y=True)